In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [2]:
prom_file_path = "../prom_integrated.parquet"

In [3]:
prom_df = pd.read_parquet(prom_file_path)

In [4]:
prom_df['successRate'] = prom_df['finishTasks'] / prom_df['jobCount']

In [5]:
prom_df.fillna(0, inplace=True)
feature_cols = [col for col in prom_df.columns if col not in ['node', 'timestamp', 'isAnomaly','node_netstat_Udp_InErrors','node_netstat_Tcp_InErrs','node_netstat_Icmp_InErrors','node_filesystem_device_error-sum','jobID','__index_level_0__']]

iforest = IsolationForest(contamination=0.028)

iforest.fit(prom_df[feature_cols])

pred = iforest.predict(prom_df[feature_cols])

pred_labels = [1 if i == -1 else 0 for i in pred]




KeyboardInterrupt: 

In [ ]:
print(classification_report(prom_df['isAnomaly'], pred_labels))

conf_mat = confusion_matrix(prom_df['isAnomaly'], pred_labels)
print('Confusion matrix:\n', conf_mat)

roc_auc = roc_auc_score(prom_df['isAnomaly'], pred_labels)
print('ROC AUC Score:', roc_auc)

fpr, tpr, _ = roc_curve(prom_df['isAnomaly'], pred_labels)
plt.figure()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

plt.savefig('roc_curve_iforest_slurm.pdf', format='pdf')

plt.show()

In [8]:
import joblib
model_filename = 'iforest_slurm_model.joblib'
joblib.dump(iforest, model_filename)

['iforest_slurm_model.joblib']